<a href="https://colab.research.google.com/github/flycye/Kabble-Tutorial/blob/main/SalesForecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# catboost - gradient boosting on decision trees
%pip install -Uq upgini catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.4/172.4 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.3/108.3 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.4/434.4 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.4/109.4 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.9/913.9 kB 14.7

In [3]:
# 80-20 rule: 80% percent for testing and 20% for training

from os.path import exists
import pandas as pd       # library dealing with dataframes

# if the downloaded zip file is not found, use the github
  # if it already exists, don't re-download it
df_path = "train.csv.zip" if exists("train.csv.zip") else "https://github.com/upgini/upgini/raw/main/notebooks/train.csv.zip"
df = pd.read_csv(df_path)   # reading csv file and inputting into df_path dataframe
df = df.sample(n = 19_000, random_state = 0)  # take a random sample of 19,000 data points

# convert store and item colns into strings
df["store"] = df["store"].astype(str)
df["item"] = df["item"].astype(str)

# convert date into pandas datetime, making sorting easier
df["date"] = pd.to_datetime(df["date"])

# sort the values in-place by date (chronological order)
df.sort_values("date", inplace = True)
df.reset_index(inplace = True, drop = True)
df.head()   # get the top five rows

,date,store,item,sales
0,2013-01-01,7,5,5
1,2013-01-01,4,9,19
2,2013-01-01,1,33,37
3,2013-01-01,3,41,14
4,2013-01-01,5,24,26


In [4]:
# put data from 2013 - 2016 into the training set, and 2017 into testing set
train = df[df["date"] < "2017-01-01"]
test = df[df["date"] >= "2017-01-01"]

In [5]:
# features are our input values and labels are our predictions

train_features = train.drop(columns = ["sales"])
train_target = train["sales"]

test_features = test.drop(columns = ["sales"])
test_target = test["sales"]

In [6]:
# enrich our features with the gini library
  # create relevant brand new features
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType

# import features enricher library into upgini enricher
enricher = FeaturesEnricher (
    search_keys = {
        "date": SearchKey.DATE,     # new data refers to same range of dates
    },

    cv = CVType.time_series         # tells enricher the type of dataset (time_series)
)

enricher.fit(train_features, train_target, eval_set[(test_features, test_target)])

HttpError: 504.0 GatewayTimeout